# ⚠ Warning

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gl/OpinionatedGeek%2Fmango-explorer/HEAD?filepath=Retrier.ipynb) _🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

# 🥭 Retrier

This notebook creates a 'retrier' context that can automatically retry failing functions.

In [ ]:
import logging
import typing

from contextlib import contextmanager


# Retrier class

This class takes a function and a maximum number of times to try running the function.

If the function succeeds, the resultant value is returned.

If the function fails by raising an exception, the function is retried.

It is retried up to the maximum number of retries. If they all fail, the last failing exception is re-raised.

This class is best used in a `with...` block using the `retry_context()` function below.

In [ ]:
class Retrier:
    def __init__(self, func: typing.Callable, retries: int) -> None:
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.func: typing.Callable = func
        self.retries: int = retries

    def run(self, *args):
        captured_exception: Exception = None
        for counter in range(self.retries):
            try:
                return self.func(*args)
            except Exception as exception:
                self.logger.info(f"Retriable call failed [{counter}] with error '{exception}'. Retrying...")
                captured_exception = exception

        raise captured_exception


# retry_context generator

This is a bit of Python 'magic' to allow using the Retrier in a `with...` block.

For example, this will call function `some_function(param1, param2)` up to `retry_count` times (7 in this case). It will only retry if the function throws an exception - the result of the first successful call is used to set the `result` variable:
```
retry_count = 7
with retry_context(some_function, retry_count) as retrier:
    result = retrier.run(param1, param2)
```

In [ ]:
@contextmanager
def retry_context(func: typing.Callable, retries: int = 3) -> typing.Iterator[Retrier]:
    yield Retrier(func, retries)


# 🏃 Running

Run a failing method, retrying it 5 times, just to show how it works in practice.

In [ ]:
if __name__ == "__main__":
    logging.getLogger().setLevel(logging.INFO)

    def _raiser(value):
        # All this does is raise an exception
        raise Exception(f"This is a test: {value}")

    # NOTE! This will fail by design, with the exception message:
    # "Exception: This is a test: ignored parameter"
    with retry_context(_raiser, 5) as retrier:
        response = retrier.run("ignored parameter")
